# config

> Utilities for loading Stata and nbstata
- order: 1

Before we can use [pystata](https://www.stata.com/python/pystata/index.html), we need to find the local Stata path (i.e., `find_path`) and then [add pystata to sys.path](https://www.stata.com/python/pystata/install.html#method-2-adding-pystata-to-sys-path) (i.e., `set_pystata_path`) so it can be imported.

The `get_config` function handles nbstata configuration, more broadly.

In [ ]:
#| default_exp config
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

## pystata configuration

In [ ]:
#| export
from nbstata.misc_utils import print_red
import os
import sys
import platform
from shutil import which
from pathlib import Path
from packaging import version
from configparser import ConfigParser, NoSectionError

In [ ]:
#| export
def _win_find_path():
    import winreg
    reg = winreg.ConnectRegistry(None, winreg.HKEY_CLASSES_ROOT)
    subkeys = [r'Stata17Do\shell\do\command',
                r'Stata18Do\shell\do\command',
                r'Stata19Do\shell\do\command']

    fpath = ''
    for subkey in subkeys:
        try:
            key = winreg.OpenKey(reg, subkey)
            fpath = winreg.QueryValue(key, None).split('"')[1]
        except FileNotFoundError:
            pass
        if fpath:
            break

    return fpath


def _mac_find_path():
    """
    Attempt to find Stata path on macOS when not on user's PATH.
    Modified from stata_kernel's original to only location "Applications/Stata". 

    Returns:
        (str): Path to Stata. Empty string if not found.
    """
    path = Path('/Applications/Stata')
    if not path.exists():
        return ''
    else:
        return str(path)

In [ ]:
#| export
def _find_path():
    if os.getenv('CONTINUOUS_INTEGRATION'):
        print('WARNING: Running as CI; Stata path not set correctly')
        return 'stata'
    if platform.system() == 'Windows':
        return _win_find_path()
    elif platform.system() == 'Darwin':
        return _mac_find_path()
    else:
        for i in ['stata-mp', 'stata-se', 'stata']:
            stata_path = which(i)
            if stata_path:
                break
        return stata_path

In [ ]:
#| hide
_find_path()

'C:\\Program Files\\Stata17\\StataMP-64.exe'

In [ ]:
#| export
def find_dir_edition():
    stata_path = _find_path()
    if stata_path is None:
        raise OSError("Stata path not found.")
    stata_dir = str(os.path.dirname(stata_path))
    stata_exe = str(os.path.basename(stata_path)).lower()

    edition = 'be'
    for e in ('se','mp'):
        if stata_exe.find(e) > -1:
            edition = e            
    return stata_dir, edition

In [ ]:
from nbstata.misc_utils import Timer

In [ ]:
#|eval: false
with Timer():
    find_dir_edition()

Elapsed time: 0.0002 seconds


In [ ]:
#| export
def set_pystata_path(path=None):
    if path == None:
        path, _ = find_dir_edition()
    if not os.path.isdir(path):
        raise OSError(path + ' is invalid')
    if not os.path.isdir(os.path.join(path, 'utilities')):
        raise OSError(path + " is not Stata's installation path")
    sys.path.append(os.path.join(path, 'utilities'))

In [ ]:
#|eval: false
with Timer():
    set_pystata_path()
    import pystata

Elapsed time: 0.0040 seconds


In [ ]:
#|eval: false
from fastcore.test import ExceptionExpected

In [ ]:
#|eval: false
with ExceptionExpected(): import sfi

In [ ]:
#| export
def launch_stata(path=None, edition=None, splash=True):
    """
    We modify stata_setup to make splash screen optional
    """
    try:
        if path == None or edition == None:
            path_found, edition_found = find_dir_edition()
            path = path_found if path==None else path
            edition = edition_found if edition==None else edition
        set_pystata_path(path)
    except OSError as err:
        pass
    import pystata
    if version.parse(pystata.__version__) >= version.parse("0.1.1"):
        # Splash message control is a new feature of pystata-0.1.1
        pystata.config.init(edition,splash=splash)
    else:
        pystata.config.init(edition)

In [ ]:
#|eval: false
with Timer():
    launch_stata(splash=False)
    pystata.config.status()

    System information
      Python version         3.10.4
      Stata version          Stata 17.0 (MP)
      Stata library path     C:\Program Files\Stata17\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = default, height = default
      graphic format         svg
Elapsed time: 0.9244 seconds


`sfi` can only be imported after Stata is launched:

In [ ]:
#|eval: false
import sfi

https://www.stata.com/python/pystata/config.html#pystata.config.set_graph_format

In [ ]:
#| export
def set_graph_format(gformat):
    import pystata
    if gformat == 'pystata':
        gformat = 'svg' # pystata default
    pystata.config.set_graph_format(gformat)

In [ ]:
#| eval: False
with Timer():
    set_graph_format('png')
pystata.config.status()

Elapsed time: 0.0000 seconds
    System information
      Python version         3.10.4
      Stata version          Stata 17.0 (MP)
      Stata library path     C:\Program Files\Stata17\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = default, height = default
      graphic format         png


In [ ]:
#| export
def set_graph_size(width, height):
    import pystata
    pystata.config.set_graph_size(width, height)

In [ ]:
#| eval: False
set_graph_size('2in', '4')
pystata.config.status()

    System information
      Python version         3.10.4
      Stata version          Stata 17.0 (MP)
      Stata library path     C:\Program Files\Stata17\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = 2.0in, height = 4in
      graphic format         png


In [ ]:
#| export
def is_graph_size_valid(size):
    import pystata
    try:
        size = pystata.config._get_figure_size_info(str(size))
        if size[0] == 'default' or size[0] < 0:
            raise ValueError('graph size is invalid')
    except ValueError:
        return False
    else:
        return True

In [ ]:
#| hide
from fastcore.test import test_eq, ExceptionExpected

In [ ]:
#| eval: False
test_eq(is_graph_size_valid('default'), False) # pystata allows, but not nbstata to avoid confusion
test_eq(is_graph_size_valid('4mi'), False)
test_eq(is_graph_size_valid('4in'), True)
test_eq(is_graph_size_valid('3.2 cm'), True)
test_eq(is_graph_size_valid('70px'), True)
test_eq(is_graph_size_valid('20'), True)
test_eq(is_graph_size_valid('-3'), False)

## nbstata configuration

In [ ]:
#| export
def _get_config_settings(cpath):
    parser = ConfigParser(
        empty_lines_in_values=False,
        comment_prefixes=('*','//'),
        inline_comment_prefixes=('//',),
    )
    parser.read(str(cpath))
    return dict(parser.items('nbstata'))

In [ ]:
#| export
class Config:
    env = {'stata_dir': None,
           'edition': 'be',
           'graph_format': 'png',
           'graph_width': '5.5in',
           'graph_height': '4in',
           'echo': 'None',
           'splash': 'False',
           'missing': '.',
          }
    valid_values_of = dict(
        graph_format={'pystata', 'svg', 'png', 'pdf'},
        echo={'True', 'False', 'None'},
        splash={'True', 'False'},
    )
    
    @property
    def splash(self):
        return False if self.env['splash']=='False' else True
    
    @property
    def noecho(self):
        return self.env['echo'] == 'None'
    
    @property
    def echo(self):
        return self.env['echo'] == 'True'
    
    def __init__(self):
        """First check if a configuration file exists. If not, try `find_dir_edition`."""
        self.errors = []
        self.config_path = None
        self._process_config_file()
        if self.env['stata_dir'] == None or self.env['edition'] == None:
            try:    
                stata_dir, stata_ed = find_dir_edition()     
            except OSError:
                pass
            else:
                self.env.update({'stata_dir': stata_dir, 'edition': stata_ed})

    def _process_config_file(self):
        global_config_path = Path(os.path.join(sys.prefix, 'etc', 'nbstata.conf'))
        user_config_path = Path('~/.nbstata.conf').expanduser()
        for cpath in (user_config_path, global_config_path):      
            if cpath.is_file():
                self._get_config_env(cpath)
                break
            
    def _get_config_env(self, cpath):
        try:
            settings = _get_config_settings(cpath)
        except ParsingError:
            print_red(f"Configuration error in {cpath}:\n"
                      "    invalid syntax")
        except DuplicateOptionError:
            print_red(f"Configuration error in {cpath}:\n"
                      "    attempted to set the same thing twice")
        except ConfigParserError:
            print_red(f"Configuration error in {cpath}")
        else:
            self.config_path = str(cpath)
            self.update(
                settings, 
                init=True, 
                error_header=f"Configuration errors in {self.config_path}:"  
            )
            
    def update(self, env, init=False, error_header="%set error(s):"):
        allowed_settings = self.env if init else set(self.env)-{'stata_dir','edition'}
        for key in list(env):
            if key not in allowed_settings:
                self.errors.append(f"    '{key}' is not a valid setting.")
                env.pop(key)
            elif (
                (key in self.valid_values_of and env[key] not in self.valid_values_of[key])
                or (key in {'graph_width', 'graph_height'} and not is_graph_size_valid(env[key]))
            ):
                self.errors.append(
                    f"    '{key}' configuration invalid. "
                    f"Reverting to: {key} = {self.env[key]}"
                )
                env.pop(key)
        self._display_and_clear_update_errors(error_header)
        for key in env:
            if not init: print(f"{key} was {self.env[key]}, is now {env[key]}")
        self.env.update(env)
  
    def _display_and_clear_update_errors(self, error_header):
        if self.errors:
            print_red(error_header)
        for message in self.errors:
            print_red(message)
        self.errors = []
        
    def display_status(self):
        import pystata
        pystata.config.status()
        print(f"""
      echo                   {self.env['echo']}
      missing                {self.env['missing']}""")

In [ ]:
config = Config()
config.env

{'stata_dir': 'C:\\Program Files\\Stata17',
 'edition': 'mp',
 'graph_format': 'png',
 'graph_width': '5.5in',
 'graph_height': '4in',
 'echo': 'None',
 'splash': 'False',
 'missing': '.'}

In [ ]:
#| eval: False
config.display_status()

    System information
      Python version         3.10.4
      Stata version          Stata 17.0 (MP)
      Stata library path     C:\Program Files\Stata17\mp-64.dll
      Stata initialized      True
      sfi initialized        True

    Settings
      graphic display        True
      graphic size           width = 2.0in, height = 4in
      graphic format         png

      echo                   None
      missing                .


In [ ]:
#| eval: False
config.update({'graph_height': '3'})

graph_height was 4in, is now 3


In [ ]:
#| eval: False
config.update({'graph_height': '-3'})

%set error(s):
    'graph_height' configuration invalid. Reverting to: graph_height = 3


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()